In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# --- Model Parameters ---
VOCAB_SIZE = 10000  # Consider the top 10,000 most frequent words
MAX_LEN = 256       # Pad/truncate news articles to 256 words
EMBEDDING_DIM = 128
RNN_UNITS = 64
LSTM_UNITS = 64
NUM_CLASSES = 46

# --- Load and Preprocess Data ---
print("Loading and preprocessing data...")
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=VOCAB_SIZE)

# Pad sequences to ensure uniform length
x_train_padded = pad_sequences(x_train, maxlen=MAX_LEN, padding='post', truncating='post')
x_test_padded = pad_sequences(x_test, maxlen=MAX_LEN, padding='post', truncating='post')

# One-hot encode the labels
y_train_one_hot = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test_one_hot = to_categorical(y_test, num_classes=NUM_CLASSES)

print(f"Shape of x_train: {x_train_padded.shape}")
print(f"Shape of y_train: {y_train_one_hot.shape}")

# --- Build SimpleRNN Model ---
def create_rnn_model():
    model = Sequential([
        Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_LEN),
        SimpleRNN(RNN_UNITS, activation='relu'),
        Dropout(0.5), # Add dropout to reduce overfitting
        Dense(NUM_CLASSES, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# --- Build LSTM Model ---
def create_lstm_model():
    model = Sequential([
        Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_LEN),
        LSTM(LSTM_UNITS),
        Dropout(0.5), # Add dropout to reduce overfitting
        Dense(NUM_CLASSES, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

rnn_model = create_rnn_model()
lstm_model = create_lstm_model()

print("--- SimpleRNN Model Summary ---")
rnn_model.summary()
print("\n--- LSTM Model Summary ---")
lstm_model.summary()

# --- Training Parameters ---
EPOCHS = 50
BATCH_SIZE = 128

print("\n--- Training SimpleRNN Model ---")
history_rnn = rnn_model.fit(
    x_train_padded, y_train_one_hot,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    verbose=1
)

print("\n--- Training LSTM Model ---")
history_lstm = lstm_model.fit(
    x_train_padded, y_train_one_hot,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    verbose=1
)

# --- Evaluate on Test Set ---
print("\n--- Evaluating Models ---")
loss_rnn, acc_rnn = rnn_model.evaluate(x_test_padded, y_test_one_hot, verbose=0)
loss_lstm, acc_lstm = lstm_model.evaluate(x_test_padded, y_test_one_hot, verbose=0)

print(f"SimpleRNN Test Accuracy: {acc_rnn*100:.2f}%")
print(f"LSTM Test Accuracy:      {acc_lstm*100:.2f}%")

Loading and preprocessing data...
Shape of x_train: (8982, 256)
Shape of y_train: (8982, 46)
--- SimpleRNN Model Summary ---


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


--- Training SimpleRNN Model ---
Epoch 1/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - accuracy: 0.1890 - loss: 3.4661 - val_accuracy: 0.3450 - val_loss: 2.4667
Epoch 2/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - accuracy: 0.3228 - loss: 2.6315 - val_accuracy: 0.3500 - val_loss: 2.4004
Epoch 3/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.3125 - loss: 2.4988 - val_accuracy: 0.3539 - val_loss: 2.3910
Epoch 4/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - accuracy: 0.3366 - loss: 2.4492 - val_accuracy: 0.3545 - val_loss: 2.3713
Epoch 5/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - accuracy: 0.3553 - loss: 2.4049 - val_accuracy: 0.3545 - val_loss: 2.3523
Epoch 6/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - accuracy: 0.3702 - loss: 2.3434 - val_accuracy: 0.3561 - val_loss: 2.3499
Epoch 7/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - accuracy: 0.3791 - loss: 2.3273 - val_accuracy: 0.3617 - val_loss: 2.3569
Epoch 8/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - accuracy: 0.3870 - loss: